# MLX Ultralytics Object Detection Training (Colab)

This notebook mirrors the MLX object detection workflow in Google Colab. It executes the following steps:
1. Clone the MLX repository.
2. Install dependencies from `requirements.txt` (patching the Ultralytics entry for HTTPS).
3. Capture the required Roboflow API credential.
4. Download and extract the DroneFace YOLOv12 dataset from Roboflow.
5. Write the provided YOLO12 model configuration.
6. Launch training through the MLX CLI.

> ⚠️ Switch your Colab runtime to **GPU** for best training speed (`Runtime → Change runtime type → GPU`).


## Step 0 · Clone the MLX repository
This pulls the public repository and switches the working directory so subsequent installs run in-place.


In [ ]:
import os
import subprocess
from pathlib import Path

REPO_URL = 'https://github.com/ralampay/mlx'
cwd = Path.cwd()
if cwd.name != 'mlx':
    repo_dir = cwd / 'mlx'
    if not repo_dir.exists():
        subprocess.run(['git', 'clone', REPO_URL], check=True)
    os.chdir(repo_dir)
print('Working directory:', Path.cwd())


## Step 1 · Install dependencies from `requirements.txt`
Colab sessions do not have SSH credentials, so we rewrite the Ultralytics dependency to use HTTPS before installing.


In [ ]:
import subprocess
import sys
from pathlib import Path

requirements_path = Path('requirements.txt')
if not requirements_path.exists():
    raise FileNotFoundError('requirements.txt not found in the repository root.')

text = requirements_path.read_text()
patched = text.replace('git+ssh://git@github.com/ralampay/ultralytics.git',
                       'git+https://github.com/ralampay/ultralytics.git')
if patched != text:
    requirements_path.write_text(patched)
    print('Patched Ultralytics dependency to use HTTPS.')
else:
    print('Ultralytics dependency already uses HTTPS.')

subprocess.run([sys.executable, '-m', 'pip', 'install', '--upgrade', 'pip'], check=True)
subprocess.run([sys.executable, '-m', 'pip', 'install', '-r', 'requirements.txt'], check=True)


## Step 2 · Install MLX locally
An editable install exposes the `mlx` CLI entrypoint.


In [ ]:
import subprocess
import sys

subprocess.run([sys.executable, '-m', 'pip', 'install', '-e', '.'], check=True)


## Step 3 · Supply the Roboflow API key
The key is required to download the DroneFace dataset and is stored only for the current Colab session.


In [ ]:
import os
import getpass

if not os.environ.get('ROBOFLOW_API_KEY'):
    os.environ['ROBOFLOW_API_KEY'] = getpass.getpass('Enter your Roboflow API Key: ')

print('Roboflow key loaded:', bool(os.environ.get('ROBOFLOW_API_KEY')))


## Step 4 · Download and extract the DroneFace YOLOv12 dataset
The Roboflow SDK pulls version 8 of the dataset and expands it under `datasets/`.


In [ ]:
from pathlib import Path
from roboflow import Roboflow
import os

api_key = os.environ.get('ROBOFLOW_API_KEY')
if not api_key:
    raise RuntimeError('ROBOFLOW_API_KEY is not set. Please rerun the previous cell.')

rf = Roboflow(api_key=api_key)
project = rf.workspace('face-detection-and-recognition-dataset').project('droneface')
dataset = project.version(8).download('yolov12', location='datasets/droneface-yolov12')
dataset_dir = Path(dataset.location)
print('Dataset ready at:', dataset_dir)
print('Sample contents:', [p.name for p in dataset_dir.iterdir()][:5])


## Step 5 · Fetch the `cad_yolo12.yaml` model configuration
Downloads the YAML from the public S3 link and stores it under `ultralytics/cfg/models/ext/cad_yolo12.yaml`.


In [ ]:
from pathlib import Path
from urllib.request import urlretrieve

destination = Path('ultralytics/cfg/models/ext/cad_yolo12.yaml')
destination.parent.mkdir(parents=True, exist_ok=True)
source_url = 'https://public-tph-yolo-models.s3.ap-southeast-1.amazonaws.com/cad_yolo12.yaml'

if destination.exists():
    print('Overwriting existing model config at:', destination)

urlretrieve(source_url, destination)
print('Model config downloaded to:', destination.resolve())


## Step 6 · Launch MLX object detection training
Adjust the hyperparameters as needed. The cell selects an available device automatically.


In [ ]:
import subprocess

try:
    dataset_dir
except NameError as exc:
    raise RuntimeError('Dataset directory not found. Please run the previous steps first.') from exc

try:
    import torch
    device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
except ImportError:
    device = 'cuda:0'

command = [
    'mlx',
    '--module', 'obj-detect',
    '--platform', 'ultralytics',
    '--action', 'train',
    '--dataset-path', str(dataset_dir),
    '--model-path', 'ultralytics/cfg/models/ext/cad_yolo12.yaml',
    '--epochs', '100',
    '--batch-size', '16',
    '--device', device,
]

print('Running:', ' '.join(command))
subprocess.run(command, check=True)
